# __WSI - ćwiczenie 2.__

### __Algorytmy ewolucyjne i genetyczne__


#### __Treść ćwiczenia__

- Celem ćwiczenia jest implementacja algorytmu ewolucyjnego z selekcją ruletkową, krzyzowaniem
jednopunktowym, mutacją gaussowską oraz sukcesją generacyjną. Następnie należy wykorzystać
implementacje do znalezienia rozwiazania dajacego minimalny koszt dla problemu opisanego w sekcji
- Nalezy znalezc zestaw hiperparametrów, który daje stosunkowo dobry wynik, a nastepnie
zbadac wpływ wybranego hiperparametru.

#### __Opis problemu__


Znalezc optymalną lokalizację dla fabryki, jeżeli korzysta ona z 4 zasobów. Zakładamy przy tym,
że:
- lokalizacja fabryki określona jest przez jej współrzedne $(x_1, x_2) \in R^2$,
- dzienne zapotrzebowanie fabryki to:
  1. $20$ jednostek zasobu $z_1$ transportowanego ze współrzednych $(1, 1)$,
  2. $10$ jednostek zasobu $z_2$ transportowanego ze współrzednych $(-0.5, 1)$,
  3. $5$ jednostek zasobu $z_3$ transportowanego ze współrzednych $(-1, -0.5)$,
  4. $10$ jednostek zasobu $z_4$ transportowanego ze współrzednych $(1, -1)$,
- koszt transportu jednostki każdego z zasobów wyliczany to $1−e^{−d_M}$, gdzie $d_M$ - to odległość
Manhattan od lokalizacji fabryki,
- celem optymalizacji jest znalezienie lokalizacji fabryki, która minimalizuje sumaryczny dzienny
koszt dostarczania zasobów do fabryki.

In [45]:
import numpy as np
from plotly import graph_objs as go
import pandas as pd
import math

RNG = np.random.default_rng()

#### __Definicja funkcji straty__

In [46]:
def manhattan(p1, p2):
    return sum([abs(p1[i] - p2[i]) for i in range(len(p1))])

def trans_cost(p1, p2):
    return 1 - math.exp(-manhattan(p1, p2))

def loss(pos):
    return (20 * trans_cost([1, 1], pos) +
            10 * trans_cost([-0.5, 1], pos) +
            5 * trans_cost([-1, -0.5], pos) +
            10 * trans_cost([1, -1], pos))

#### __Algorytm ewolucyjny__

__Selekcja ruletkowa:__ 
- Funkcja prawdopodobieństwa wyboru osobnika $i$: $p_i = \frac{g_i} {\sum\limits_{j=1}^{N}g_j}$
- Dla problemu minimalizacji zamieniam funkcję straty w funkcję zysku $g(x) = f_{max} - f(x)$
- Najgorszy osobnik ma zerowe szanse przetrwania

In [47]:
def select_roulette(population):
    min_grades = [loss(x) for x in population]
    max_grade = max(min_grades)
    min_to_max = [max_grade - f for f in min_grades]
    sum_grades = sum(min_to_max)
    max_grades = [g/sum_grades for g in min_to_max]
    return RNG.choice(population, len(population), p=max_grades)

__Krzyowanie jednopunktowe:__

In [48]:
def one_point_cross():
    pass

__Mutacja Gaussowska:__

In [49]:
def mutate_gauss():
    pass

__Sukcesja generacyjna:__
- Zaimplementowana wyłącznie w celu ewentualnej zamiany na inną metodę sukcesji

In [50]:
def replace():
    pass

__Algorytm ewolucyjny:__

In [51]:
def evo(start_pop, num_iters, pop_size):
    # selection
    # crossover
    # mutation
    # replacement
    pass